In [1]:
import os
import torch
import torch.nn as nn
import torch.distributed as dist
from torch.cuda.amp import GradScaler, autocast
from torch.utils.data.distributed import DistributedSampler
import wandb
import argparse
from model import SpeechEmotionClassifier
from data import RAVDESSDataset
from utils import read_config, split_dataset
os.environ["CUDA_VISIBLE_DEVICES"] = "6"

/home/ec2-user/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
config_file = "config/whisper_based.yaml"
config = read_config(config_file)

In [3]:
if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'
config = read_config(config_file)
full_dataset = RAVDESSDataset(config.data_path, config.model_name)
_, _, test_dataset = split_dataset(full_dataset, config)

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=config.batch_size, shuffle=False)

In [4]:
model = SpeechEmotionClassifier(config.model_name, config.class_num)

In [5]:
model_path = f'./results/whisper_emotion_epoch_{config.epochs}_{config.model_name}/shanda.pth'
state_dict = torch.load(model_path)
# Remove the "module." prefix from the keys
new_state_dict = {key.replace("module.", ""): value for key, value in state_dict.items()}

# model.load_state_dict(torch.load(model_path))

KeyError: 'model_state_dict'

In [8]:
model.load_state_dict(new_state_dict)

KeyError: 'module'

In [11]:
print(checkpoint.keys())

odict_keys(['module.whisper_model.encoder.conv1.weight', 'module.whisper_model.encoder.conv1.bias', 'module.whisper_model.encoder.conv2.weight', 'module.whisper_model.encoder.conv2.bias', 'module.whisper_model.encoder.embed_positions.weight', 'module.whisper_model.encoder.layers.0.self_attn.k_proj.weight', 'module.whisper_model.encoder.layers.0.self_attn.v_proj.weight', 'module.whisper_model.encoder.layers.0.self_attn.v_proj.bias', 'module.whisper_model.encoder.layers.0.self_attn.q_proj.weight', 'module.whisper_model.encoder.layers.0.self_attn.q_proj.bias', 'module.whisper_model.encoder.layers.0.self_attn.out_proj.weight', 'module.whisper_model.encoder.layers.0.self_attn.out_proj.bias', 'module.whisper_model.encoder.layers.0.self_attn_layer_norm.weight', 'module.whisper_model.encoder.layers.0.self_attn_layer_norm.bias', 'module.whisper_model.encoder.layers.0.fc1.weight', 'module.whisper_model.encoder.layers.0.fc1.bias', 'module.whisper_model.encoder.layers.0.fc2.weight', 'module.whispe